In [3]:
#most recent code
#step1-Install libraries

!pip install earthengine-api geemap
!pip install -U geemap
!pip install geopandas
!pip install pycrs

import ee
import geemap
from google.colab import drive

#step2- initialize EE
ee.Authenticate()
ee.Initialize(project='ee-youngclick')

#step 3- Mount drive
drive.mount('/content/drive')

#step4- Define aoi coordinates
aoi_coordinates = [
    [-88.6954, 35.7933],
    [-89.3381, 35.8000],
    [-89.3134, 34.9850],
    [-88.6981, 34.9918]
]

#Step 5- Create geometry from the coordinate aoi
aoi = ee.Geometry.Polygon(aoi_coordinates)

#step6- Filter sentinel-2 harmonized collection by cloud cover and date
collection = (
    ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")
    .filterBounds(aoi)
    .filterDate("2022-12-01", "2023-02-28")
    .filter(ee.Filter.lt("CLOUDY_PIXEL_PERCENTAGE", 10))
    .sort("CLOUDY_PIXEL_PERCENTAGE")
)

#Step 7- Create a median composite image for the coordinate aoi
med_sen = collection.median().clip(aoi)

#Step8- Calculate ndvi for the coordinate aoi
ndvi_aoi = med_sen.normalizedDifference(["B8", "B4"]).rename("NDVI")

#Step9-Call stream shp from gee project to create clipping geometry for clipping
clip_geometry = ee.FeatureCollection("projects/ee-youngclick/assets/bond_stpl_polyline").geometry()

#Step10-Apply 50m buffer areound the stream study area shp
buffered_clip_geometry = clip_geometry.buffer(50)

#Step10-clip the ndvi result using the buffered study area aoi
ndvi_clipped = ndvi_aoi.clip(buffered_clip_geometry)

# Step 11-visualization
rgb_vis_params = {
    "min": 0,
    "max": 3000,
    "bands": ["B4", "B3", "B2"],
}

ndvi_vis_params = {
    "min": -1,
    "max": 1,
    "palette": ["#FFFF00", "#ADFF2F", "#008000", "#006400"]  # codes for ndvi- yellow(low ndvi) to dark green(high ndvi)
}




# Step12-display the mediancomposite  image, ndvi Layer, study area aoi, and study area clipped ndvi layer on the map
Map = geemap.Map(center=[35.2, -88.9], zoom=9)
Map.addLayer(med_sen, rgb_vis_params, "median composite image of Sentinel-2)")
Map.addLayer(ndvi_aoi, ndvi_vis_params, "ndvi of coordinate defined aoi")
Map.addLayer(aoi, {"color": "yellow"}, "Study area aoi boundary")
Map.addLayer(buffered_clip_geometry, {"color": "blue"}, "buffered study area boundary (Bond Creek)")
Map.addLayer(ndvi_clipped, ndvi_vis_params, "Study area clipped ndvi")


# Step 13- display map
Map


  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.0/56.0 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 456.0/456.0 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 46.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.9/108.9 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.5/12.5 MB 109.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.5/46.5 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 52.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.8/101.8 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 51.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 84.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 49.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━

Map(center=[35.2, -88.9], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI…